# global_attention_model(BahdanauAttention)

In [2]:
import tensorflow as tf
from itertools import chain

### 데이터

In [3]:
source = [['<sos>', '타임', '아웃', '<eos>'],
         ['<sos>', '너무', '자랑스러워', '<eos>']]

target = [['<sos>', 'time', 'out', '<eos>'],
          ['<sos>', 'i', 'am', 'proud', 'of', 'you', '<eos>']]

dic = list(set(list(chain(*source)) + list(chain(*target)))) + ["<pad>"]

### 배치 구성

In [4]:
fedic_encoder_seq_len_except_pad = [len(i) for i in source]
fedic_encoder_x = []
for sentence in source:
    if len(sentence) == max(fedic_encoder_seq_len_except_pad):
        fedic_encoder_x.append([dic.index(i) for i in sentence])
    else:
        sentence += ['<pad>']*(max(fedic_encoder_seq_len_except_pad) - len(sentence))
        fedic_encoder_x.append([dic.index(i) for i in sentence])

fedic_decoder_seq_len_except_pad = [len(i)-1 for i in target]
fedic_decoder_x = []
real_label = []
for sentence in target:
    if (len(sentence)-1) == max(fedic_decoder_seq_len_except_pad):
        fedic_decoder_x.append([dic.index(i) for i in sentence[:-1]])
        real_label.append([dic.index(i) for i in sentence[1:]])
    else:
        sentence += ['<pad>']*(max(fedic_decoder_seq_len_except_pad) - len(sentence) + 1)
        fedic_decoder_x.append([dic.index(i) for i in sentence[:-1]])
        real_label.append([dic.index(i) for i in sentence[1:]])

In [5]:
fedic_encoder_x

[[2, 4, 3, 11], [2, 0, 12, 11]]

In [6]:
fedic_encoder_seq_len_except_pad

[4, 4]

In [7]:
fedic_decoder_x

[[2, 8, 10, 11, 13, 13], [2, 5, 6, 1, 9, 7]]

In [8]:
fedic_decoder_seq_len_except_pad

[3, 6]

In [9]:
real_label

[[8, 10, 11, 13, 13, 13], [5, 6, 1, 9, 7, 11]]

### 파라미터 설정

In [10]:
number_of_document = 2
number_of_encoder_word = None
number_of_decoder_word = None
lstm_hidden_size = 10
voca_size = len(dic)
word_embedding_size = 2
max_gradient_norm = 0.1
learning_rate = 1e-4

### label

In [11]:
y = tf.placeholder(dtype=tf.int32, shape=[number_of_document, number_of_decoder_word])

### encoder

In [12]:
encoder_x = tf.placeholder(dtype=tf.int32, shape=[number_of_document, number_of_encoder_word])
source_sequence_lengths = tf.placeholder(dtype=tf.int32, shape=[number_of_document])
embedding_matrix = tf.get_variable(name="embeding_matrix", shape=[voca_size, word_embedding_size],
                                   dtype=tf.float32, initializer=tf.truncated_normal_initializer())
encoder_emb_inp = tf.nn.embedding_lookup(params=embedding_matrix, ids=encoder_x, name="encoder_emb_inp")
encoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=lstm_hidden_size,
                                       initializer=tf.contrib.layers.variance_scaling_initializer())
encoder_outputs, encoder_final_hidden_state = tf.nn.dynamic_rnn(cell=encoder_cell , inputs=encoder_emb_inp,
                                                                sequence_length=source_sequence_lengths, dtype=tf.float32,
                                                                scope="encoder_LSTM")

### attention

In [13]:
attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(lstm_hidden_size, encoder_outputs, memory_sequence_length=source_sequence_lengths)

### decoder

In [14]:
decoder_x = tf.placeholder(dtype=tf.int32, shape=[number_of_document, number_of_decoder_word])
decoder_lengths = tf.placeholder(dtype=tf.int32, shape=[number_of_document])
decoder_emb_inp = tf.nn.embedding_lookup(params=embedding_matrix, ids=decoder_x, name="decoder_embedded_x")
helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, decoder_lengths)
decoder_lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units=lstm_hidden_size, initializer=tf.contrib.layers.variance_scaling_initializer())
decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_lstm_cell, attention_mechanism, attention_layer_size=lstm_hidden_size)
decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, decoder_cell.zero_state(batch_size=number_of_document,dtype=tf.float32), output_layer=tf.layers.Dense(voca_size))
decoder_outputs, decoder_final_hidden_state, decoder_final_sequence_length = tf.contrib.seq2seq.dynamic_decode(decoder) # decoder_outputs=logits
logits = decoder_outputs.rnn_output

### loss

In [15]:
crossent = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y,voca_size), logits=logits)
train_loss = tf.reduce_sum(crossent)

### calculate and clip gradients

In [16]:
params = tf.trainable_variables()
gradients = tf.gradients(train_loss, params)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm) # gradient exploding 방지

### optimization

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate)
update_step = optimizer.apply_gradients(zip(clipped_gradients, params))

### training

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    _, print_loss = sess.run([update_step,train_loss], feed_dict={encoder_x:fedic_encoder_x,
                                                                  source_sequence_lengths:fedic_encoder_seq_len_except_pad,
                                                                  decoder_x:fedic_decoder_x,
                                                                  decoder_lengths:fedic_decoder_seq_len_except_pad,
                                                                  y:real_label})
    if i%500 == 0 :
        a1, a2 = sess.run([tf.argmax(tf.nn.softmax(logits), axis=2), train_loss], feed_dict={encoder_x: fedic_encoder_x,
                                                                                             source_sequence_lengths: fedic_encoder_seq_len_except_pad,
                                                                                             decoder_x: fedic_decoder_x,
                                                                                             decoder_lengths: fedic_decoder_seq_len_except_pad, y: real_label})
        print_list = []
        for i in a1:
            sen = ""
            for j in i:
                sen += dic[j]+" "
            print_list.append(sen)

        print("sentence : ", print_list, "loss : ", a2)

sentence :  ['자랑스러워 자랑스러워 <pad> <pad> <pad> <pad> ', '자랑스러워 자랑스러워 <pad> <pad> 타임 <pad> '] loss :  31.30599
sentence :  ['<pad> <pad> <pad> <pad> <pad> <pad> ', '<pad> <pad> <pad> <pad> <pad> <pad> '] loss :  25.820177
sentence :  ['time <pad> <pad> <pad> <pad> <pad> ', 'time <eos> <pad> <pad> <eos> <eos> '] loss :  19.85854
sentence :  ['time out <pad> <pad> <pad> <pad> ', 'time proud proud of <eos> <eos> '] loss :  14.188705
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i proud proud of <eos> <eos> '] loss :  8.310126
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i am proud of you <eos> '] loss :  4.2367907
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i am proud of you <eos> '] loss :  1.6728804
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i am proud of you <eos> '] loss :  0.5391512
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i am proud of you <eos> '] loss :  0.15911245
sentence :  ['time out <eos> <pad> <pad> <pad> ', 'i am proud of you <eos> '] loss : 